# T1546.004 - Unix Shell Configuration Modification
Adversaries may establish persistence through executing malicious commands triggered by a user’s shell. User [Unix Shell](https://attack.mitre.org/techniques/T1059/004)s execute several configuration scripts at different points throughout the session based on events. For example, when a user opens a command-line interface or remotely logs in (such as via SSH) a login shell is initiated. The login shell executes scripts from the system (<code>/etc</code>) and the user’s home directory (<code>~/</code>) to configure the environment. All login shells on a system use /etc/profile when initiated. These configuration scripts run at the permission level of their directory and are often used to set environment variables, create aliases, and customize the user’s environment. When the shell exits or terminates, additional shell scripts are executed to ensure the shell exits appropriately. 

Adversaries may attempt to establish persistence by inserting commands into scripts automatically executed by shells. Using bash as an example, the default shell for most GNU/Linux systems, adversaries may add commands that launch malicious binaries into the <code>/etc/profile</code> and <code>/etc/profile.d</code> files.(Citation: intezer-kaiji-malware)(Citation: bencane blog bashrc) These files typically require root permissions to modify and are executed each time any shell on a system launches. For user level permissions, adversaries can insert malicious commands into <code>~/.bash_profile</code>, <code>~/.bash_login</code>, or <code>~/.profile</code> which are sourced when a user opens a command-line interface or connects remotely.(Citation: anomali-rocke-tactics)(Citation: Linux manual bash invocation) Since the system only executes the first existing file in the listed order, adversaries have used <code>~/.bash_profile</code> to ensure execution. Adversaries have also leveraged the <code>~/.bashrc</code> file which is additionally executed if the connection is established remotely or an additional interactive shell is opened, such as a new tab in the command-line interface.(Citation: Tsunami)(Citation: anomali-rocke-tactics)(Citation: anomali-linux-rabbit)(Citation: Magento) Some malware targets the termination of a program to trigger execution, adversaries can use the <code>~/.bash_logout</code> file to execute malicious commands at the end of a session. 

For macOS, the functionality of this technique is similar but may leverage zsh, the default shell for macOS 10.15+. When the Terminal.app is opened, the application launches a zsh login shell and a zsh interactive shell. The login shell configures the system environment using <code>/etc/profile</code>, <code>/etc/zshenv</code>, <code>/etc/zprofile</code>, and <code>/etc/zlogin</code>.(Citation: ScriptingOSX zsh)(Citation: PersistentJXA_leopitt)(Citation: code_persistence_zsh)(Citation: macOS MS office sandbox escape) The login shell then configures the user environment with <code>~/.zprofile</code> and <code>~/.zlogin</code>. The interactive shell uses the <code>~/.zshrc</code> to configure the user environment. Upon exiting, <code>/etc/zlogout</code> and <code>~/.zlogout</code> are executed. For legacy programs, macOS executes <code>/etc/bashrc</code> on startup.

## Atomic Tests

### Atomic Test #1 - Add command to .bash_profileAdds a command to the .bash_profile file of the current user
**Supported Platforms:** macos, linux#### Attack Commands: Run with `sh`
```sh
echo 'echo "Hello from Atomic Red Team T1546.004" > /tmp/T1546.004' >> ~/.bash_profile
```

In [ ]:
Invoke-AtomicTest T1546.004 -TestNumbers 1

#### Cleanup: 
```sh
head -n '-2' ~/.bash_profile > /tmp/T1546.004
mv /tmp/T1546.004 ~/.bash_profile
```

In [ ]:
Invoke-AtomicTest T1546.004 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Add command to .bashrcAdds a command to the .bashrc file of the current user
**Supported Platforms:** macos, linux#### Attack Commands: Run with `sh`
```sh
echo 'echo "Hello from Atomic Red Team T1546.004" > /tmp/T1546.004' >> ~/.bashrc
```

In [ ]:
Invoke-AtomicTest T1546.004 -TestNumbers 2

#### Cleanup: 
```sh
head -n '-2' ~/.bashrc > /tmp/T1546.004
mv /tmp/T1546.004 ~/.bashrc
```

In [ ]:
Invoke-AtomicTest T1546.004 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Append to the system shell profileAn adversary may wish to establish persistence by executing malicious commands from the systems /etc/profile every time "any" user logs in.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
echo '# Hello from Atomic Red Team T1546.004' >> /etc/profile
```

In [ ]:
Invoke-AtomicTest T1546.004 -TestNumbers 3

#### Cleanup: 
```sh
sed -i "s/# Atomic Red Team was here! T1546.004//" /etc/profile
```

In [ ]:
Invoke-AtomicTest T1546.004 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Append commands user shell profileAn adversary may wish to establish persistence by executing malicious commands from the users ~/.profile every time the "user" logs in.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
echo '# Atomic Red Team was here... T1546.004' >> ~/.profile
```

In [ ]:
Invoke-AtomicTest T1546.004 -TestNumbers 4

#### Cleanup: 
```sh
sed -i "s/# Atomic Red Team was here... T1546.004//" ~/.profile
```

In [ ]:
Invoke-AtomicTest T1546.004 -TestNumbers 4 -Cleanup

### Atomic Test #5 - System shell profile scriptsAn adversary may wish to establish persistence by adding commands into any of the script files in the /etc/profile.d/ directory, which are executed every time "any" user logs in.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
echo '# Atomic Red Team was here... T1546.004' >> /etc/profile.d/bash_completion.sh
```

In [ ]:
Invoke-AtomicTest T1546.004 -TestNumbers 5

#### Cleanup: 
```sh
sed -i "s/# Atomic Red Team was here... T1546.004//" /etc/profile.d/bash_completion.sh
```

In [ ]:
Invoke-AtomicTest T1546.004 -TestNumbers 5 -Cleanup

## Detection
While users may customize their shell profile files, there are only certain types of commands that typically appear in these files. Monitor for abnormal commands such as execution of unknown programs, opening network sockets, or reaching out across the network when user profiles are loaded during the login process.

Monitor for changes to <code>/etc/profile</code> and <code>/etc/profile.d</code>, these files should only be modified by system administrators. MacOS users can leverage Endpoint Security Framework file events monitoring these specific files.(Citation: ESF_filemonitor) 

For most Linux and macOS systems, a list of file paths for valid shell options available on a system are located in the <code>/etc/shells</code> file.
